In [2]:
import pandas as pd
import requests
import bs4
import re

CSV = False

# Scraping a few career pages to check for similarities

Goal:

**Attempt to find semi-universal code to scrape job postings from different websites**
